Copy reserve price distribution from round 1.

Cases:
- if reserve price < b1:
    - profit = 1000 - b1 (same as round 1)
- elif reserve price < b2:
    - our b2 is higher than the avg of all b2:
        - profit = 1000 - b2 (same as round 1)
    - else:
        - profit = p(1000 - b2) where p = (1000 – average bid) / (1000 – your bid)

Note: b2 being equal to avg_b2 results in the same return regardless of computation method

In [1]:
import numpy as np

In [2]:
# Define the range and probabilities
min_value = 900
max_value = 1000
num_values = max_value - min_value + 1
probabilities = np.linspace(0, 1, num_values)

# Normalize probabilities to sum to 1
probabilities /= probabilities.sum()

Our submission in round 1:
- b1: 952
- b2: 978

Based on the seashell profit made, this seems to be close to the optimal solution. We should expect most competitors to be aware of this and take this into consideration for round 4.

Assuming our b2 is always higher than the avg of all b2s, this should still be the optimal solution. However, rational competitors will be adjusting their b2 based on this information. The constraints in round 4 shouldn't be significantly affecting b1s.

In [6]:
### Simulation

def simulate(no_samples, no_simulations, avg_b2):

    r1, r2, = 0, 0

    for x in range(no_simulations):

        # Generate random samples
        samples = np.random.choice(np.arange(min_value, max_value + 1), size=no_samples, p=probabilities)

        max_profit = 0
        best_bid = [0,0]

        # simulate all possible bids

        for b1 in range(900, 1001):
            for b2 in range(b1, 1001):

                profit = 0

                # iterate through sample birds
                for reserve in samples:
                    # compare reserve price to bids
                    if reserve < b1:
                        profit += 1000-b1
                    elif reserve < b2:
                        # compare b2 and avg_b2
                        if b2 > avg_b2:
                            profit += 1000-b2
                        else:
                            profit += ((1000-avg_b2) / (1000-b2)) * (1000-b2)

                if profit > max_profit:
                    max_profit = profit
                    best_bid = [b1, b2]
        
        r1+=best_bid[0]
        r2+=best_bid[1]

    print("Avg optimal bids: ", r1/no_simulations, r2/no_simulations)

In [7]:
simulate(1000, 100, 950)
simulate(1000, 100, 960)
simulate(1000, 100, 970)
simulate(1000, 100, 980)
simulate(1000, 100, 990)

Avg optimal bids:  952.25 978.16
Avg optimal bids:  952.01 978.16
Avg optimal bids:  952.37 978.34
Avg optimal bids:  953.23 980.49
Avg optimal bids:  959.98 990.0


In [10]:
simulate(1000, 100, 978)
simulate(1000, 100, 980)
simulate(1000, 100, 982)
simulate(1000, 100, 984)
simulate(1000, 100, 986)
simulate(1000, 100, 998)

Avg optimal bids:  952.93 979.3
Avg optimal bids:  953.42 980.59
Avg optimal bids:  954.31 982.15
Avg optimal bids:  955.91 984.06
Avg optimal bids:  957.83 986.01
Avg optimal bids:  964.99 998.0


In [11]:
simulate(1000, 1000, 978)
simulate(1000, 1000, 980)
simulate(1000, 1000, 982)
simulate(1000, 1000, 984)
simulate(1000, 1000, 986)
simulate(1000, 1000, 998)

Avg optimal bids:  952.635 979.29
Avg optimal bids:  953.402 980.545
Avg optimal bids:  954.521 982.185
Avg optimal bids:  955.651 984.036
Avg optimal bids:  957.201 986.003
Avg optimal bids:  965.115 998.0


It seems like the optimal b1 is also higher if avg_b2>b2. This is because we compute lower than expected profits (scaling factor: p) when reserve price >b1 but <b2 and b2<avg_b2. It becomes optimal to increase b1 to trade on higher reserve prices without losing out because of the scaling factor. Can verify with a closed form solution?

Soln ( only integer bids accepted):
- b1: 952 (+(avg_b2-978)/2 if avg_b2 > 978)
- b2: 978 (+(avg_b2-978) if avg_b2 > 978)

Our submission depends on what we expect avg_b2 to be. I'm assuming that optimal b2 from round 1 + (1|2|3) will be popular submissions. Let's compute how profit gets penalized based on how far off our b2 is from avg_b2.

In [20]:
### Simulation

def profit(no_samples, no_simulations, b1, b2, avg_b2):

    profit_sum = 0

    for x in range(no_simulations):

        # Generate random samples
        samples = np.random.choice(np.arange(min_value, max_value + 1), size=no_samples, p=probabilities)

        # simulate all possible bids

        for reserve in samples:
            if reserve < b1:
                profit_sum += 1000-b1
            elif reserve < b2:
                if b2 > avg_b2:
                    profit_sum += 1000-b2
                else:
                    profit_sum += ((1000-avg_b2) / (1000-b2)) * (1000-b2)

    print(f"b1: {b1}, b2: {b2}, avg_b2: {avg_b2}, Avg profit: {profit_sum/no_simulations}")

In [29]:
profit(1000, 10000, 952, 978, 978)
profit(1000, 10000, 953, 979, 978)
profit(1000, 10000, 954, 980, 978)
profit(1000, 10000, 954, 981, 978)
profit(1000, 10000, 952, 979, 978)
profit(1000, 10000, 952, 980, 978)
profit(1000, 10000, 954, 981, 979)
profit(1000, 10000, 954, 982, 979)
profit(1000, 10000, 952, 980, 980)
profit(1000, 10000, 953, 980, 980)
profit(1000, 10000, 952, 981, 980)
profit(1000, 10000, 953, 981, 980)
profit(1000, 10000, 954, 981, 980)
profit(1000, 10000, 953, 982, 980)
profit(1000, 10000, 954, 982, 980)
profit(1000, 10000, 954, 983, 980)
profit(1000, 10000, 954, 981, 981)
profit(1000, 10000, 952, 980, 982)
profit(1000, 10000, 954, 981, 982)
profit(1000, 10000, 952, 982, 982)
profit(1000, 10000, 954, 982, 982)
profit(1000, 10000, 952, 983, 982)
profit(1000, 10000, 953, 983, 982)
profit(1000, 10000, 952, 984, 982)
profit(1000, 10000, 953, 984, 982)
profit(1000, 10000, 954, 984, 982)
profit(1000, 10000, 955, 984, 982)
profit(1000, 10000, 955, 985, 982)

b1: 952, b2: 978, avg_b2: 978, Avg profit: 19906.085
b1: 953, b2: 979, avg_b2: 978, Avg profit: 19910.8992
b1: 954, b2: 980, avg_b2: 978, Avg profit: 19883.0436
b1: 954, b2: 981, avg_b2: 978, Avg profit: 19841.0613
b1: 952, b2: 979, avg_b2: 978, Avg profit: 19895.1933
b1: 952, b2: 980, avg_b2: 978, Avg profit: 19878.0832
b1: 954, b2: 981, avg_b2: 979, Avg profit: 19835.957
b1: 954, b2: 982, avg_b2: 979, Avg profit: 19778.4428
b1: 952, b2: 980, avg_b2: 980, Avg profit: 19860.8168
b1: 953, b2: 980, avg_b2: 980, Avg profit: 19882.7286
b1: 952, b2: 981, avg_b2: 980, Avg profit: 19807.3751
b1: 953, b2: 981, avg_b2: 980, Avg profit: 19824.9539
b1: 954, b2: 981, avg_b2: 980, Avg profit: 19836.2891
b1: 953, b2: 982, avg_b2: 980, Avg profit: 19760.3801
b1: 954, b2: 982, avg_b2: 980, Avg profit: 19768.9708
b1: 954, b2: 983, avg_b2: 980, Avg profit: 19671.7136
b1: 954, b2: 981, avg_b2: 981, Avg profit: 19835.2828
b1: 952, b2: 980, avg_b2: 982, Avg profit: 19133.9226
b1: 954, b2: 981, avg_b2: 982,

### Submission:
- Lowest Bid: 954
- Highest Bid: 981

This seems like the best solution if avg_bid is <= 981. If avg_bid is 982, [954,982] would be a better solution being 1.4% more profitable.